In [ ]:
from main_calculation import *

In [ ]:
a, df, b, c = calculate('https://www.bbc.co.uk/food/recipes/honeyandmarmaladegla_93186')

In [ ]:
from scraper_parser import *

In [ ]:
a,b,c = get_ingredients_url('https://www.bbc.co.uk/food/recipes/honeyandmarmaladegla_93186')

In [ ]:
d,e,f = get_ingredients_url('https://www.bbc.co.uk/food/recipes/loaded_aubergine_burgers_44288')

In [ ]:
### FOR TEST
h,i,j = get_ingredients_url('https://www.bbc.co.uk/food/recipes/great_british_31279')

In [ ]:
def pre_process_summary(summary):

    # Remove punctuation
    for punctuation in string.punctuation:
        summary = str(summary).replace(punctuation, '')

    # Lower text
    summary = summary.lower()

    # Stopwords
    stop_words = set(stopwords.words('english'))
    summary_tokenized = word_tokenize(summary)
    text = [w for w in summary_tokenized if not w in stop_words]
    summary = ' '.join(text)
    return summary


In [ ]:
a1 = pre_process_summary(a)

In [ ]:
d1 = pre_process_summary(d)

In [ ]:
h1 = pre_process_summary(h)

In [ ]:
def get_match_and_score(summary_vector):

    scoreseries = []

    for j, _ in enumerate(category_summaries):

        cosine_sum = 1 - spatial.distance.cosine(
            summary_vector.toarray(), category_summary_vectors[j, :].toarray())

        scoreseries.append(cosine_sum)

    matchscore = max(scoreseries)
    match = category_list[scoreseries.index(matchscore)]

    return match, matchscore

In [ ]:
1 - spatial.distance.cosine(a2.toarray(), h2.toarray())

In [ ]:
with open('matching_objects2.pk', 'rb') as handle:
    matching_objects_dict = cPickle.load(handle)
    vectorizer = matching_objects_dict['vectorizer']

In [ ]:
a2 = vectorizer.transform([a1])
d2 = vectorizer.transform([d1])
h2 = vectorizer.transform([h1])

In [ ]:
1 - spatial.distance.cosine(d2.toarray(), h2.toarray())

In [ ]:
k,l,m = get_ingredients_url('https://www.bbc.co.uk/food/recipes/spinach_and_mushroom_31250')

In [ ]:
k1 = pre_process_summary(k)
k2 = vectorizer.transform([k1])

In [ ]:
1 - spatial.distance.cosine(d2.toarray(), k2.toarray())

In [1]:
from scraper_parser import *

In [14]:
def parse_recipe_ingredients(ingredient_list):

    """Wrapper around parse_ingredient so we can call it on an ingredient list"""
    sentences = tokenizer.tokenize(ingredient_list)
    sentences = [sent.strip('\n') for sent in sentences]
    names = []
    qtys = {}
    units = {}
    our_punctuation = '!"#$%&\'())*+:;<=>?@[\\]^_`{|}~'
    our_units = ['gram', 'milliliters', 'thigh', 'wing', 'breast']

    for i in range(len(sentences)):
        for punctuation in our_punctuation:
        # cleaning for common issues
            sentences[i] = sentences[i].replace(punctuation, '')

        sentences[i] = sentences[i].replace('can', '')
        sentences[i] = sentences[i].replace('package', '')
        sentences[i] = sentences[i].replace('container', '')
        sentences[i] = sentences[i].replace('eggs eggs', 'eggs')
        sentences[i] = sentences[i].replace('⅓', '.33')
        sentences[i] = sentences[i].replace('½', '.5')
        sentences[i] = sentences[i].replace('¼', '.25')
        sentences[i] = sentences[i].replace('¾', '.75')
        sentences[i] = sentences[i].replace('tsp', 'teaspoon')
        sentences[i] = sentences[i].replace('tbsp', 'tablespoon')
        sentences[i] = sentences[i].replace('large', '')
        sentences[i] = sentences[i].replace('medium', '')
        sentences[i] = sentences[i].replace('small', '')
        sentences[i] = sentences[i].replace('kg', '000g')
        sentences[i] = sentences[i].replace('aubergine', 'eggplant')
        sentences[i] = sentences[i].replace('free-range', '')

        if re.search("\dg", sentences[i]) is not None:
            sentences[i] = sentences[i].replace("g", "gram", 1)


        parsed_ingredient = parse_ingredient(sentences[i])
        
        ## filling names
        if 'name' in parsed_ingredient[0].keys():

            tmp = parsed_ingredient[0]['name']
            useless_quantifiers = ['oz', 'fl', 'ounce']

            try:
                names = re.search("[^\d]*$", tmp).group(0)
            except:
                names = tmp

        else:
            names = np.nan
            
        ##filling units
        #import pdb; pdb.set_trace()
        for unit in our_units:
            if unit in parsed_ingredient[0]['input']:
                units[names] = unit
            elif 'unit' in parsed_ingredient[0].keys():
                units[names] = parsed_ingredient[0]['unit']
        try: 
            units[names]
        except:
            units[names] = 'unit'

        ##filling quantities
        if re.search("\dg", sentences[i]) is not None:
            try:
                qtys[names] = re.search("\d+(?=\s*g)", parsed_ingredient[0]['input']).group(0)
            except:
                pass
        elif re.search("\dkg", sentences[i]) is not None:
            try:
                qtys[names] = re.search("\d+(?=\s*kg)", parsed_ingredient[0]['input']).group(0)
            except:
                pass
        elif 'qty' in parsed_ingredient[0].keys():
            qtys[names] = parsed_ingredient[0]['qty']
        else:
            try:
                qtys[names] = float(parsed_ingredient[0]['input'][:3])
            except:
                qtys[names] = np.nan
    
    
    ## collating all elements together
    names_list = []
    units_list = []
    qtys_list = []
    for key in units.keys():
        names_list.append(key)
        qtys_list.append(qtys[key])
        units_list.append(units[key])
        
    final_df = pd.DataFrame(list(zip(qtys_list, units_list, names_list)), columns = ['qty', 'unit', 'name'])

    final_df = final_df[final_df['name'] != '']
    final_df = final_df[final_df['name'].notna()]
    final_df = final_df[final_df['unit'].notna()]

    final_df.loc[final_df['unit'] == 'teaspoon', 'qty'] = 0
    final_df.loc[final_df['unit'] == 'teaspoon', 'unit'] = 'gram'
    final_df.loc[final_df['qty'].astype(str) == 'nan', 'qty'] = 1

    final_df.reset_index()

    return final_df

In [13]:
ingredient_list,b,c = get_ingredients_url('https://www.bbc.co.uk/food/recipes/stuffedchickenbreast_89240')

In [15]:
test_df = parse_recipe_ingredients(ingredient_list)

In [16]:
names_list = []
units_list = []
qtys_list = []
for key in units.keys():
    names_list.append(key)
    qtys_list.append(qtys[key])
    units_list.append(units[key])

test_df = pd.DataFrame(list(zip(qtys_list, units_list, names_list)), columns = ['qty', 'unit', 'name'])

NameError: name 'units' is not defined

In [6]:
qtys

['breast',
 'thighs',
 'unit',
 'unit',
 'unit',
 'tablespoon',
 'tablespoon',
 'gram',
 'unit',
 'gram',
 'unit',
 'unit',
 'tablespoon',
 'gram',
 'tablespoon']

## Improving calculator with new units

In [5]:
import pandas as pd
data = pd.read_csv('data/Final_conversion_table.csv')
data = data.drop(['Average tablespoon (grams)', 'Source', 'Average Unit (if applicable) (grams)', 'Source.1'], axis = 1)
data.columns = ['name','tablespoon','unit','wing','breast','drumstick','thigh','chop','fillet','ghg']

In [6]:
data

,name,tablespoon,unit,wing,breast,drumstick,thigh,chop,fillet,ghg
0,Alcohol,0.01000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5000
1,Almonds,0.01000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.4100
2,Anise,0.01000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7300
3,Apples,NaN,0.080,NaN,NaN,NaN,NaN,NaN,NaN,0.4000
4,Apricot,0.01000,0.060,NaN,NaN,NaN,NaN,NaN,NaN,0.6315
...,...,...,...,...,...,...,...,...,...,...
128,Wheat & Rye (Bread),0.00981,0.053,NaN,NaN,NaN,NaN,NaN,NaN,1.3000
129,White bean,0.01000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.7850
130,Wine,0.01475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6000
131,Wine vinegar,0.01000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.1700


In [20]:
from main_calculation import *

===== printing directory =======
===== printing content directory =======


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/georgesdelrieu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/georgesdelrieu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/georgesdelrieu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/georgesdelrieu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [64]:
from matching import *

In [65]:
with open('matching_objects2.pk', 'rb') as handle:
    matching_objects_dict = cPickle.load(handle)
    vectorizer = matching_objects_dict['vectorizer']
    df_wiki_match_scores = matching_objects_dict['df_wiki_match_scores']
    category_list = matching_objects_dict['category_list']
    category_summary_vectors = matching_objects_dict[
        'category_summary_vectors']
    category_summaries = matching_objects_dict['category_summaries']


In [66]:
vectorizer.transform(category_summaries[:132])

<132x3970 sparse matrix of type '<class 'numpy.float64'>'
	with 13502 stored elements in Compressed Sparse Row format>

In [68]:
def get_google_match(ingredient):

    try:
        ingredient, url, url_base = get_google_cse_result(ingredient)
    except:
        return 'nomatch', 0

    pageid = get_pageid_from_base(url_base)

    pagesummary = get_summary_from_id(pageid)

    processed_summary = pre_process_summary(pagesummary)

    summary_vector = vectorizer.transform([processed_summary])
    
    return summary_vector

In [69]:
get_google_match('ham')

<1x3970 sparse matrix of type '<class 'numpy.float64'>'
	with 34 stored elements in Compressed Sparse Row format>

In [88]:
def get_match_and_score(summary_vector):
    category_summary_vectors = vectorizer.transform(category_summaries)
    scoreseries = []

    for j, _ in enumerate(category_summaries):
        cosine_sum = 1 - spatial.distance.cosine(
            summary_vector.toarray(), category_summary_vectors[j, :].toarray())

        scoreseries.append(cosine_sum)

    matchscore = max(scoreseries)
    match = category_list[scoreseries.index(matchscore)]

    return match, matchscore

In [89]:
category_summary_vectors

<132x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 13502 stored elements in Compressed Sparse Row format>

In [90]:
get_match_and_score(get_google_match('ham'))

(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)
(133, 3970)
(1, 3970)


('Pork Meat', 0.21647900132201947)

In [40]:
get_match_and_score(get_google_match('ham'))

('Pork Meat', 0.21647900132201947)